# MTAOS addAberration

Notebooks to show how the addAberration command in MTAOS works.

(using OFC code before Tiago's rewrite in April 2021)

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np

from lsst.ts.ofc.ctrlIntf.OFCCalculationFactory import OFCCalculationFactory
from lsst.ts.wep.Utility import CamType
from lsst.ts.ofc.Utility import getConfigDir, DofGroup, InstName
from lsst.ts.ofc.ctrlIntf.FWHMSensorData import FWHMSensorData
from lsst.ts.wep.ctrlIntf.MapSensorNameAndId import MapSensorNameAndId
from lsst.ts.wep.ctrlIntf.SensorWavefrontError import SensorWavefrontError
from lsst.ts.wep.Utility import FilterType
from lsst.ts.wep.bsc.CamFactory import CamFactory
from lsst.ts.MTAOS.CollOfListOfWfErr import CollOfListOfWfErr

In [2]:
getConfigDir()

'/home/bxin/lsst_stack/ts_ofc/policy'

In [3]:
#watch out, do not use from lsst.ts.wep.Utility.CamType.ComCam
ofc = OFCCalculationFactory.getCalculator(InstName.COMCAM, None)

In [4]:
ofc.ztaac.dataShare.getDofIdx()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [5]:
comcam = CamFactory.createCam(CamType.ComCam)

/home/bxin/lsst_stack/ts_wep/python/lsst/ts/wep/bsc/ComCam.py:34: FutureWarning: Gen2 Butler has been deprecated (LsstCamMapper). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  super(ComCam, self).__init__(LsstCamMapper().camera)


In [6]:
detectorList = comcam.getWfsCcdList()

In [7]:
sensorIdList = ofc.ztaac.mapSensorNameToId(detectorList)

In [8]:
intrinsic_zk = ofc.ztaac.dataShare.getIntrinsicZk(FilterType.REF, ofc.ztaac.dataShare.getFieldIdx(detectorList))

## If you pass Intrinsic aberrations to OFC, you get all corrections = 0

In [9]:
listOfWfErr = []
for sensorId, annularZernikePoly in zip(sensorIdList, intrinsic_zk):
    #print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr.append(sensorWavefrontError)

In [10]:
(
    m2HexapodCorrection,
    cameraHexapodCorrection,
    m1m3Correction,
    m2Correction,
) = ofc.calculateCorrections(listOfWfErr)

In [11]:
print(m2HexapodCorrection.getCorrection())
print(cameraHexapodCorrection.getCorrection())
print(m1m3Correction.getZForces())
print(m2Correction.getZForces())

(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Add 1um z7 (We tell OFC that we measure intrinsic - (1um z7), so that it adds 1um z7 to go back to intrinsic

In [12]:
add_coma = np.array(intrinsic_zk, copy=True)
add_coma[:, 7-4] -= 1 #z4 has index 0

listOfWfErr_coma = []
for sensorId, annularZernikePoly in zip(sensorIdList, add_coma):
    #print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_coma.append(sensorWavefrontError)
    
ofc = OFCCalculationFactory.getCalculator(InstName.COMCAM, None)

(
    m2HexapodCorrection_cx1,
    cameraHexapodCorrection_cx1,
    m1m3Correction_cx1,
    m2Correction_cx1,
) = ofc.calculateCorrections(listOfWfErr_coma)

In [13]:
print(m2HexapodCorrection_cx1.getCorrection())
print(cameraHexapodCorrection_cx1.getCorrection())
print(m1m3Correction_cx1.getZForces())
print(m2Correction_cx1.getZForces())

(-0.05491948079476481, -71.8523602612489, -0.16912072688512814, 0.003293368961278743, 2.2676602850106736e-06, 0.0)
(0.025150067821742523, 37.96873264721965, -0.008264539229012947, 0.00011663343158918227, 6.679176306937749e-07, 0.0)
[ 1.80604649e-02 -2.28653820e-02 -2.78958470e-02 -2.29876715e-03
  2.71762902e-02  4.98860752e-02  5.30540370e+00  5.58094341e+00
  4.46250156e+00  2.58714223e+00  6.42585435e-01 -9.55541778e-01
  1.15409302e+01  1.12497049e+01  9.23964800e+00  5.91740508e+00
  2.22455424e+00 -1.15694359e+00 -2.40849502e+00  1.46352660e+01
  1.22675876e+01  8.31374163e+00  3.52358705e+00 -1.24463344e+00
 -3.70137508e+00  1.31810208e+01  1.17674648e+01  8.16786179e+00
  3.15621799e+00 -2.12836414e+00 -5.26547944e+00  7.16293880e+00
  4.52300704e+00  2.51386719e-01 -4.50111134e+00 -6.59903821e+00
 -1.07200509e+00 -2.62840955e+00 -3.10125963e+00 -8.20224182e+00
 -9.76642508e+00 -9.37503030e+00 -8.88022064e+00  5.19863025e+00
  5.48261576e+00  4.46030384e+00  2.63015538e+00  6.7

In [14]:
(
    m2HexapodCorrection_cx2,
    cameraHexapodCorrection_cx2,
    m1m3Correction_cx2,
    m2Correction_cx2,
) = ofc.calculateCorrections(listOfWfErr_coma)

print(np.array(m2HexapodCorrection_cx2.getCorrection())[:5]/np.array(m2HexapodCorrection_cx1.getCorrection())[:5])

[1.35653437 1.78411044 1.2802434  1.70934537 1.49842256]


### This is because the default control strategy is "x00"

In [15]:
ofc.ztaac.dataShare.xRef

'x00'

In [16]:
dofIdx = ofc.ztaac.dataShare.getDofIdx()
print(ofc.ztaac.optCtrl.getState0(dofIdx))
print(ofc.ztaac.optCtrl.getState(dofIdx))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[ 2.16515694e-01  7.45001633e-02 -1.28192546e+02 -2.02662179e+01
 -1.22324880e-02  9.04618131e-02 -3.90641994e-02  6.26129880e+01
  1.82867534e-02 -2.87636267e-03 -2.06499997e-03 -2.97754236e-03
 -1.14015241e-03 -5.73029598e-03  8.61930665e-04 -2.15364361e-04
 -1.21787455e-01  8.07079750e-05  1.05782488e-05  6.83577137e-05
 -3.26392478e-05  2.40825301e-05 -3.09977646e-05  7.01907222e-04
  1.03628070e-05  9.92030986e-03  8.01354006e-05 -1.94247268e-04
 -4.32246368e-06 -5.79635275e-06 -4.77743092e-03 -2.62055528e-03
 -8.24667360e-04 -6.84649210e-03  2.33999229e-04 -1.32821214e-01
  4.62806684e-02  3.45093041e-05 -1.41707803e-04 -3.02489046e-05
 -4.52796488e-05 -1.56063529e-05  8.04855697e-05  1.95730293e-05
  3.71909769e-04  1.03984610e-07  8.45550548e-08 -1.93550467e-06
  6.67533042e-04  1.88646674e-03]


In [17]:
add_coma = np.array(intrinsic_zk, copy=True)
add_coma[:, 7-4] -= 1 #z4 has index 0

listOfWfErr_coma = []
for sensorId, annularZernikePoly in zip(sensorIdList, add_coma):
    #print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_coma.append(sensorWavefrontError)
    
ofc = OFCCalculationFactory.getCalculator(InstName.COMCAM, None)

(
    m2HexapodCorrection_cx1,
    cameraHexapodCorrection_cx1,
    m1m3Correction_cx1,
    m2Correction_cx1,
) = ofc.calculateCorrections(listOfWfErr_coma)

state0 = np.zeros(ofc.ztaac.optCtrl.getNumOfState0())
ofc.ztaac.optCtrl.setState(state0)

(
    m2HexapodCorrection_cx2,
    cameraHexapodCorrection_cx2,
    m1m3Correction_cx2,
    m2Correction_cx2,
) = ofc.calculateCorrections(listOfWfErr_coma)

print(np.array(m2HexapodCorrection_cx2.getCorrection())[:5]/np.array(m2HexapodCorrection_cx1.getCorrection())[:5])

[1. 1. 1. 1. 1.]


### What happens if we set control strategy to "x0"

In [18]:
ofc.ztaac.dataShare.xRef 

'x00'

In [19]:
add_coma = np.array(intrinsic_zk, copy=True)
add_coma[:, 7-4] -= 1 #z4 has index 0

listOfWfErr_coma = []
for sensorId, annularZernikePoly in zip(sensorIdList, add_coma):
    #print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_coma.append(sensorWavefrontError)
    
ofc = OFCCalculationFactory.getCalculator(InstName.COMCAM, None)
ofc.ztaac.dataShare.xRef = 'x0'

(
    m2HexapodCorrection_cx1,
    cameraHexapodCorrection_cx1,
    m1m3Correction_cx1,
    m2Correction_cx1,
) = ofc.calculateCorrections(listOfWfErr_coma)

#state0 = np.zeros(ofc.ztaac.optCtrl.getNumOfState0())
#ofc.ztaac.optCtrl.setState(state0)

(
    m2HexapodCorrection_cx2,
    cameraHexapodCorrection_cx2,
    m1m3Correction_cx2,
    m2Correction_cx2,
) = ofc.calculateCorrections(listOfWfErr_coma)

print(np.array(m2HexapodCorrection_cx2.getCorrection())[:5]/np.array(m2HexapodCorrection_cx1.getCorrection())[:5])

[2. 2. 2. 2. 2.]


In [20]:
(
    m2HexapodCorrection,
    cameraHexapodCorrection,
    m1m3Correction,
    m2Correction,
) = ofc.resetOfcState()

In [21]:
print(m2HexapodCorrection.getCorrection())
print(cameraHexapodCorrection.getCorrection())
print(m1m3Correction.getZForces())
print(m2Correction.getZForces())

(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
